In [ ]:
import matplotlib.pyplot as plt

# Access the specific data and headers
data_entry = nested_csv_data['24K CF+2400tex BF hybrid']['50psi']['5']
data_array = data_entry['data']
headers = data_entry['headers']

# Assuming the second column is now the x-axis and the first column is the y-axis
x = data_array[:, 1]  # Swapping the indices for x
y = data_array[:, 0]  # Swapping the indices for y

# Create the plot
plt.figure(figsize=(10, 6))
plt.plot(x, y, marker='o', linestyle='-')  # Line plot with markers
plt.title(f'Plot of {headers[0]} vs {headers[1]}')
plt.xlabel(headers[1])  # Use the second header for the x-axis label
plt.ylabel(headers[0])  # Use the first header for the y-axis label
plt.xlim(left=0)
plt.ylim(bottom=0)


# Display the plot
plt.show()
import matplotlib.pyplot as plt
import numpy as np

# Access the data for the specific material type
material_data = nested_csv_data['12K CF']

# Initialize a dictionary to store the pressures and their corresponding average maximums and standard deviations
pressure_data = {}

# Iterate over each pressure level in the material's data
for pressure, csv_data in material_data.items():
    max_values = []
    
    # Iterate over each CSV's data within this pressure level
    for identifier, data_entry in csv_data.items():
        data_array = data_entry['data']
        column_1_max = np.max(data_array[:, 0])  # Max of column 1
        max_values.append(column_1_max)
    
    # Calculate the average and standard deviation of the max values for this pressure
    average_max = np.mean(max_values)
    stddev_max = np.std(max_values)
    pressure_data[pressure] = (average_max, stddev_max)

# Handle sorting where 'cont' can't be converted to int
def custom_pressure_sort(p):
    try:
        return int(p.split('psi')[0])
    except ValueError:
        return float('inf')  # Place non-numeric pressures at the end

sorted_pressures = sorted(pressure_data.keys(), key=custom_pressure_sort)

# Filter out non-numeric pressures for plotting and fitting
numeric_pressures = [p for p in sorted_pressures if p != 'cont']
sorted_averages = [pressure_data[p][0] for p in numeric_pressures]
sorted_stddevs = [pressure_data[p][1] for p in numeric_pressures]
pressure_numeric = [int(p.split('psi')[0]) for p in numeric_pressures]

# 3rd order polynomial fit
coefficients = np.polyfit(pressure_numeric, sorted_averages, 3)
poly_fit = np.poly1d(coefficients)

# Format polynomial equation for the legend
equation = f'Fit: y = {coefficients[0]:.2f}x³ + {coefficients[1]:.2f}x² + {coefficients[2]:.2f}x + {coefficients[3]:.2f}'

# Create a smoother x range for the polynomial curve
x_smooth = np.linspace(min(pressure_numeric), max(pressure_numeric), 500)
y_smooth = poly_fit(x_smooth)

# Create a scatter plot with error bars
plt.figure(figsize=(10, 6))
plt.errorbar(pressure_numeric, sorted_averages, yerr=sorted_stddevs, fmt='o', color='blue', ecolor='lightgray', elinewidth=3, capsize=0)
plt.plot(x_smooth, y_smooth, 'r--', label=equation)

# Set custom ticks to reflect the sorted pressures as labels
plt.xticks(pressure_numeric, [p + ' psi' for p in map(str, pressure_numeric)], rotation=45)  # Apply the original pressure labels

# Plot settings
plt.title('Mean Failure Load (N) vs Pressure (PSI)')
plt.xlabel('Pressure (PSI)')
plt.ylabel('Mean Failure Load (N)')
plt.legend()

# Display the plot
plt.show()
import matplotlib.pyplot as plt
import numpy as np

# Assuming nested_csv_data is organized with each key as a material type
# and each value as the data structured similarly to the previous example.

# Determine the number of materials and prepare the subplot grid
num_materials = len(nested_csv_data)
num_columns = 3  # You can adjust this based on how wide your display is
num_rows = (num_materials + num_columns - 1) // num_columns  # Calculate rows needed

# Create a large figure to hold all subplots
fig, axes = plt.subplots(num_rows, num_columns, figsize=(15, 5 * num_rows))
axes = axes.flatten()  # Flatten the axis array for easier iteration

# Loop over each material type and its data
for idx, (material, material_data) in enumerate(nested_csv_data.items()):
    pressure_data = {}

    # Process data for each pressure within this material
    for pressure, csv_data in material_data.items():
        if 'psi' in pressure:  # Process only data with 'psi' in the pressure string
            max_values = []

            # Extract maximums from each CSV's data
            for identifier, data_entry in csv_data.items():
                data_array = data_entry['data']
                column_1_max = np.max(data_array[:, 0])
                max_values.append(column_1_max)

            # Compute the average and standard deviation
            average_max = np.mean(max_values)
            stddev_max = np.std(max_values)
            pressure_data[pressure] = (average_max, stddev_max)

    # Check if there are numeric pressure data available to plot
    if pressure_data:
        # Sort pressures numerically and ignore non-numeric values
        sorted_pressures = [p for p in pressure_data.keys()]
        sorted_pressures.sort(key=lambda x: int(x.split('psi')[0]))
        sorted_averages = [pressure_data[p][0] for p in sorted_pressures]
        sorted_stddevs = [pressure_data[p][1] for p in sorted_pressures]
        pressure_numeric = [int(p.split('psi')[0]) for p in sorted_pressures]

        if pressure_numeric:  # Ensure there are numeric pressures to fit
            # Fit a 2nd order polynomial
            coefficients = np.polyfit(pressure_numeric, sorted_averages, 2)
            poly_fit = np.poly1d(coefficients)

            # Generate equation string for legend
            equation = f'{material}: y = {coefficients[0]:.2f}x² + {coefficients[1]:.2f}x + {coefficients[2]:.2f}'

            # Create smooth x values for plotting the curve
            x_smooth = np.linspace(min(pressure_numeric), max(pressure_numeric), 500)
            y_smooth = poly_fit(x_smooth)

            # Plot data and fit curve
            ax = axes[idx]
            ax.errorbar(pressure_numeric, sorted_averages, yerr=sorted_stddevs, fmt='o', color='blue', ecolor='lightgray', elinewidth=3, capsize=0)
            ax.plot(x_smooth, y_smooth, 'r--', label=equation)
            ax.set_title(material)
            ax.set_xlabel('Pressure (PSI)')
            ax.set_ylabel('Mean Failure Load (N)')
            ax.legend()
            ax.set_xlim(left=0)  # Set the left x-axis limit to 0
            ax.set_ylim(bottom=0)  # Set the bottom y-axis limit to 0
        else:
            print(f"No numeric pressure data available for plotting for material: {material}")
    else:
        print(f"No pressure data available for material: {material}")

# Hide unused axes if there are fewer materials than subplots
for i in range(idx + 1, len(axes)):
    axes[i].set_visible(False)

# Adjust layout to prevent label overlap
plt.tight_layout()
plt.show()
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Set font to Times New Roman and size 14 globally
plt.rcParams['font.family'] = 'Times New Roman'
plt.rcParams['font.size'] = 14

# Assuming 'structured_df' is your DataFrame
# Grouping by Material and Pressure to get the required statistics
agg_funcs = {
    'Original Mass': ['mean', 'std'],
    'Spliced Mass': ['mean', 'std']
}
grouped = structured_df.groupby(['Material', 'Pressure']).agg(agg_funcs)
grouped.columns = ['Average Original Mass', 'Std Original Mass', 'Average Spliced Mass', 'Std Spliced Mass']

# Calculate the Normalized Mass Loss using the aggregated averages
grouped['Average Normalized Mass Loss'] = ((grouped['Average Original Mass'] - grouped['Average Spliced Mass']) / grouped['Average Original Mass']) * 100

# Calculate Std Dev of Normalized Mass Loss using propagation of uncertainty
grouped['Std Dev Normalized Mass Loss'] = np.sqrt(
    (-100 * (grouped['Average Spliced Mass'] / grouped['Average Original Mass']**2) * grouped['Std Original Mass'])**2 +
    (100 * (1 / grouped['Average Original Mass']) * grouped['Std Spliced Mass'])**2
)

grouped.reset_index(inplace=True)
grouped['Pressure'] = grouped['Pressure'].astype(str)  # Ensure pressure is string for manipulation

for material, material_data in grouped.groupby('Material'):
    filtered_data = material_data[~material_data['Pressure'].str.contains('cont', case=False)]
    sorted_data = filtered_data.sort_values(by='Pressure', key=lambda x: np.array([int(p.rstrip('psi').strip()) for p in x]))

    pressures = sorted_data['Pressure'].tolist()
    avg_normalized_loss = sorted_data['Average Normalized Mass Loss'].tolist()
    std_dev_normalized_loss = sorted_data['Std Dev Normalized Mass Loss'].tolist()

    x = np.arange(len(pressures))
    width = 0.35

    fig, ax = plt.subplots(figsize=(5, 4))
    bars1 = ax.bar(x, avg_normalized_loss, width*2, color='skyblue', edgecolor='black', label='Average Normalized Mass Loss', yerr=std_dev_normalized_loss)

    ax.set_title(f'Normalized Mass Loss (%) for {material}', fontweight='bold', fontsize=14)
    ax.set_xlabel('Pressure')
    ax.set_ylabel('Normalized Mass Loss (%)')
    ax.set_xticks(x)
    ax.set_xticklabels([label.rstrip('psi').strip() for label in pressures])
    # ax.set_ylim(bottom=0)

    plt.tight_layout()
    plt.show()
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.font_manager import FontProperties

# Set font to Times New Roman and size 14 globally
plt.rcParams['font.family'] = 'Times New Roman'
plt.rcParams['font.size'] = 14

# Target displacement interval
desired_interval = 9043

# Assuming calculate_average_max_stiffness is defined and nested_csv_data is structured
stiffness_results_by_material = {}
closest_intervals = {}

for material in nested_csv_data.keys():
    stiffness_data, intervals = calculate_average_max_stiffness(nested_csv_data, material)
    # Find the interval closest to the desired value
    closest_interval = min(intervals, key=lambda x: abs(x - desired_interval))
    closest_intervals[material] = closest_interval
    stiffness_results_by_material[material] = stiffness_data
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.font_manager import FontProperties

# Set global properties for Times New Roman font and figure size
plt.rcParams['font.family'] = 'Times New Roman'
plt.rcParams['font.size'] = 14
plt.rcParams['figure.figsize'] = (5, 4)

def calculate_average_max_stiffness(data, material):
    if material not in data:
        print("Material not found in the data.")
        return {}, []

    stiffness_results = {}
    
    # Define displacement intervals
    displacement_intervals = list(range(301, 16301, 800))

    for pressure, datasets in data[material].items():
        print(f"Processing pressure: {pressure}")

        all_max_stiffness = {interval: [] for interval in displacement_intervals}

        total_files = len(datasets)
        file_count = 0

        for identifier in datasets:
            file_count += 1
            print(f"Processing file {file_count} of {total_files} under pressure {pressure}.")

            array = datasets[identifier]['data']
            force = array[:, 0]
            displacement = array[:, 1]

            for interval in displacement_intervals:
                max_stiffness = float('-inf')
                for i in range(len(displacement) - interval):
                    delta_force = force[i + interval] - force[i]
                    delta_displacement = displacement[i + interval] - displacement[i]

                    if delta_displacement != 0:
                        stiffness = delta_force / delta_displacement
                        if stiffness > max_stiffness:
                            max_stiffness = stiffness

                all_max_stiffness[interval].append(max_stiffness)

        avg_max_stiffness_per_interval = {interval: np.mean(max_vals) for interval, max_vals in all_max_stiffness.items()}
        stiffness_results[pressure] = avg_max_stiffness_per_interval
        print(f"Completed processing for pressure {pressure}.")

    return stiffness_results, displacement_intervals

# Assuming nested_csv_data is a dictionary containing all your materials and their data
all_materials = nested_csv_data.keys()

for material in all_materials:
    stiffness_data, intervals = calculate_average_max_stiffness(nested_csv_data, material)

    plt.figure()
    for pressure, avg_stiffness_data in stiffness_data.items():
        stiffness_values = [avg_stiffness_data[interval] for interval in intervals]
        intervals_array = np.array(intervals) * 0.000494701
        plt.plot(intervals_array, stiffness_values, marker='o', linestyle='-', label=f'Pressure: {pressure}')

    plt.title(f'Stiffness Analysis for {material}', fontweight='bold')
    plt.xlabel('Displacement Interval (mm)')
    plt.ylabel('Average Maximum Stiffness')
    plt.grid(False)
    # Set legend font size smaller and make it draggable
    legend = plt.legend(prop=FontProperties(size=11.5))
    legend.set_draggable(True)
    plt.tight_layout()
    plt.show()
import os
import pandas as pd
import re

def load_csv_files_with_headers(base_dir):
    # Nested dictionary to hold NumPy arrays and headers
    data = {}
    
    # Regex to extract pressure information (e.g., "50psi") or "cont" as continuous data
    pressure_pattern = re.compile(r'(\d+psi|cont)')
    
    # Walk through the directory structure
    for root, dirs, files in os.walk(base_dir):
        parts = root.split(os.sep)
        # Try to find the pressure part in each part of the path
        pressure = None
        material = None
        for part in parts:
            match = re.search(pressure_pattern, part)
            if match:
                pressure = match.group(1)
                material = parts[parts.index(part) - 1]  # Assumes material type is just before the pressure part
                break
        
        if pressure and material:
            # Initialize the dictionary hierarchy if not already present
            if material not in data:
                data[material] = {}
            if pressure not in data[material]:
                data[material][pressure] = {}
            
            for file in files:
                # Check if the file is a CSV
                if file.endswith('.csv'):
                    identifier = file.replace('.csv', '')
                    
                    # Load CSV into a DataFrame
                    file_path = os.path.join(root, file)
                    df = pd.read_csv(file_path)
                    
                    # Convert DataFrame to NumPy array and extract headers
                    array = df.to_numpy()
                    headers = df.columns.tolist()
                    
                    # Store array and headers in nested dictionary
                    data[material][pressure][identifier] = {'headers': headers, 'data': array}
                    print(f"Loaded and converted CSV to NumPy array with headers for {material} under {pressure} with identifier {identifier}")
    
    return data

# Specify the base directory
base_dir = '/Users/jamesdavidson/Documents/ECCM21 Technical Paper/ECCM21_Test_Data'

# Load CSV files into nested NumPy arrays with headers
nested_csv_data = load_csv_files_with_headers(base_dir)

# Example: Print all loaded materials and pressures with headers
print("All loaded materials and pressures:")
for material in nested_csv_data:
    print(f"Material: {material}")
    for pressure in nested_csv_data[material]:
        print(f"  Pressure: {pressure}")
        for identifier in nested_csv_data[material][pressure]:
            print(f"    Identifier: {identifier}")
            print(f"      Headers: {nested_csv_data[material][pressure][identifier]['headers']}")
import pandas as pd

# Load the Excel file
file_path = '/Users/jamesdavidson/Documents/ECCM21 Technical Paper/ECCM21_Mass_Loss_Data/Mass_Loss.xlsx'
data = pd.read_excel(file_path, header=None)

# Function to parse and structure the DataFrame
def structure_data(data):
    materials = data.iloc[0, :].tolist()
    pressures = data.iloc[1, :].tolist()
    types = data.iloc[2, :].tolist()

    structured_data = []
    # Iterate over each column triplet
    for i in range(0, len(data.columns), 3):
        for row in range(3, data.shape[0]):
            # Handle integers in pressures by converting to string and stripping any spaces
            material = str(materials[i]).strip()
            pressure = str(pressures[i]).strip()  # Convert to string to use strip()
            specimen_data = {
                "Material": material,
                "Pressure": pressure,
                "Original Mass": float(data.iloc[row, i]) if types[i] == 'original' else None,
                "Spliced Mass": float(data.iloc[row, i+1]) if types[i+1] == 'spliced' else None,
                "Mass Loss": float(data.iloc[row, i+2]) if types[i+2] == 'mass loss' else None
            }
            structured_data.append(specimen_data)
    return pd.DataFrame(structured_data)

# Convert list of dicts to DataFrame
structured_df = structure_data(data)
structured_df.dropna(subset=['Original Mass', 'Spliced Mass', 'Mass Loss'], inplace=True)

# Calculate average and standard deviation of mass loss for each material-pressure combination
grouped = structured_df.groupby(['Material', 'Pressure'])['Mass Loss'].agg(['mean', 'std'])
print(grouped)

In [ ]:
%matplotlib widget
import matplotlib.pyplot as plt
import numpy as np

# assuming nested_csv_data is structured and accessible
num_materials = len(nested_csv_data)

# set font to times new roman and size 12 globally
plt.rc('font', family='times new roman', size=14)

for material, material_data in nested_csv_data.items():
    pressure_data = {}
    cont_value = none

    for pressure, csv_data in material_data.items():
        max_values = [np.max(data_entry['data'][:, 0]) for identifier, data_entry in csv_data.items()]
        average_max = np.mean(max_values)
        stddev_max = np.std(max_values)
        pressure_data[pressure] = (average_max, stddev_max)

        if pressure == 'cont':
            cont_value = average_max

    numeric_pressures = {k: v for k, v in pressure_data.items() if 'psi' in k}
    sorted_pressures = sorted(numeric_pressures.keys(), key=lambda x: int(x.split('psi')[0]))
    sorted_averages = [numeric_pressures[p][0] for p in sorted_pressures]
    sorted_stddevs = [numeric_pressures[p][1] for p in sorted_pressures]
    pressure_labels = [f"{int(p.split('psi')[0])}" for p in sorted_pressures]

    x = np.arange(len(sorted_pressures))
    width = 0.35

    fig, ax = plt.subplots(figsize=(5, 4))

    if cont_value is not none:
        bars1 = ax.bar(x - width/2, sorted_averages, width, yerr=sorted_stddevs, capsize=3, color='skyblue', edgecolor='black', label='mean failure load (n)')
        recovery_percentages = [(value / cont_value) * 100 for value in sorted_averages]
        ax2 = ax.twinx()
        bars2 = ax2.bar(x + width/2, recovery_percentages, width, color='grey', edgecolor='black', label='recovery %')

        ax2.set_ylabel('recovery %')
        ax2.set_ylim(0, 100)

        lines, labels = ax.get_legend_handles_labels()
        lines2, labels2 = ax2.get_legend_handles_labels()
        legend = ax2.legend(lines + lines2, labels + labels2, loc='upper center', bbox_to_anchor=(0.5, 0.1), fancybox=true, shadow=true, ncol=1, fontsize=11)
        legend.set_draggable(true)

    else:
        bars = ax.bar(x, sorted_averages, width*2, yerr=sorted_stddevs, capsize=3, color='skyblue', edgecolor='black', label='mean failure load (n)')
        ax.set_ylim(bottom=0)

        legend = ax.legend(loc='upper center', bbox_to_anchor=(0.5, 0.1), fancybox=true, shadow=true, ncol=2, fontsize=11.5)
        legend.set_draggable(true)

    # set bold title with specific font settings
    ax.set_title(material, fontdict={'weight': 'bold', 'size': 14})
    ax.set_xlabel('pressure (psi)')
    ax.set_ylabel('mean failure load (n)')
    ax.set_xticks(x)
    ax.set_xticklabels(pressure_labels)

    plt.tight_layout()
    plt.show()

In [ ]:
# visualize data with recovery percentages
for material, stiffness_data in stiffness_results_by_material.items():
    closest_interval = closest_intervals[material]
    pressures = list(stiffness_data.keys())

    # exclude 'cont' and sort pressures numerically
    filtered_pressures = [p for p in pressures if 'cont' not in p.lower()]
    sorted_pressures = sorted(filtered_pressures, key=lambda x: int(x.rstrip('psi').strip()))

    # extract stiffness values and calculate recovery if 'cont' value exists
    if 'cont' in pressures:
        cont_value = stiffness_data['cont'][closest_interval]
        average_stiffnesses = [stiffness_data[pressure][closest_interval] for pressure in sorted_pressures]
        recovery_percentages = [(value / cont_value) * 100 for value in average_stiffnesses]
    else:
        print(f"no 'cont' value for recovery calculation in material {material}.")
        continue

    x = np.arange(len(sorted_pressures))
    width = 0.35

    fig, ax = plt.subplots(figsize=(5, 4))
    bars1 = ax.bar(x - width/2, average_stiffnesses, width, color='skyblue', edgecolor='black', label='average maximum stiffness')

    ax.set_title(f'average maximum stiffness and recovery for {material}', fontweight='bold', fontsize=14)
    ax.set_xlabel('pressure')
    ax.set_ylabel('average maximum stiffness (n/mm)')
    ax.set_xticks(x)
    ax.set_xticklabels([label.rstrip('psi').strip() for label in sorted_pressures])
    
    # plotting recovery percentages on a second y-axis
    ax2 = ax.twinx()
    bars2 = ax2.bar(x + width/2, recovery_percentages, width, color='grey', edgecolor='black', label='recovery %')
    ax2.set_ylabel('recovery %')
    ax2.set_ylim(0, 150)  # assuming recovery could be more than 100%

    lines, labels = ax.get_legend_handles_labels()
    lines2, labels2 = ax2.get_legend_handles_labels()
    ax.legend(lines + lines2, labels + labels2, loc='upper left')

    plt.tight_layout()
    plt.show()